In [47]:
# !pip install grpcio==1.33.2
# !pip install google-ads==12.0.0 # New API Version
try:
    import argparse
    import sys
    import os
    import pandas as pd
    import numpy as np
    import json
    from datetime import datetime, timedelta, date
    #from dateutil import relativedelta
    import requests

    import concurrent.futures
    #from multiprocessing.dummy import Pool
    from multiprocessing import cpu_count
    from functools import partial
    import time

    import csv
    import re
    from io import StringIO

    import boto3
    s3 = boto3.client('s3')
    s3.download_file("fury-data-apps", "marketing-utils/pzenone/utils.py","utils.py")
    import utils

    from google.ads.googleads.client import GoogleAdsClient
    from google.ads.googleads.errors import GoogleAdsException

    import base64

    bq_credential = base64.b64decode(os.environ['SECRET_AUTH_BIGQUERY_MODEL'])
    sub = utils.Substatus(bq_credential)
    sub.subscriber('start', "ALL", "ML", "AUDIENCIAS", "REPORTING")

    ######################### Datos de user list - criterion ID
    def gads_campaign_report(account_id, filter_interval=('DURING','LAST_7_DAYS')):
        global GADS_CLIENT
        global GADS_SERVICE

        enum_channel_types = GADS_CLIENT.get_type('AdvertisingChannelTypeEnum', version="v8").AdvertisingChannelType

        MAX_RETRY_ATTEMPTS = 3
        BACKOFF_FACTOR = 30
        BOOL_RESPONSE = False

        if filter_interval[0].upper() == 'DURING':
            campaign_data_range = "DURING {}".format(filter_interval[1])
        elif filter_interval[0].upper() == 'BETWEEN':
            campaign_data_range = "BETWEEN {}".format(filter_interval[1])


        campaign_data_query = """
                SELECT user_list.name, user_list.id,user_list.type,
                       user_interest.name,user_interest.user_interest_id, 
                       campaign_criterion.criterion_id 
                FROM campaign_criterion
        """

        campaign_data_rows = []
        campaign_data_errors = []
        cdr_append = campaign_data_rows.append
        retry_attempt = 0

        try:
            gads_response = GADS_SERVICE.search(customer_id = account_id, query=campaign_data_query)
            BOOL_RESPONSE = True

            for gads_row in gads_response:
                try:
                    campaign_data = {
                        "USER_LIST":gads_row.user_list.name,
                        "USER_ID":gads_row.user_list.id,
                        "USER_TYPE":gads_row.user_list.type_,
                        "USER_INTEREST_NAME":gads_row.user_interest.name,
                        "USER_INTEREST_ID":gads_row.user_interest.user_interest_id, 
                        "CRITERION_ID":gads_row.campaign_criterion.criterion_id
                    }

                    cdr_append(campaign_data)

                except Exception as e:
                    print("Error occured handling response")
                    print(e)
                    continue

        except GoogleAdsException as ex:
            error_type = '{error_status}'.format(error_status=ex.error.code().name)
            error_messages = ''
            print(
                f"Request with ID '{ex.request_id}' failed with status "
                f"'{ex.error.code().name}' and includes the following errors:"
            )
            for error in ex.failure.errors:
                error_message = 'Error Message: {message}'
                print(f"\tError with message '{error.message}'.")
                if error.location:
                    error_message += ' - '
                    for field_path_element in error.location.field_path_elements:
                        error_message += '\n On Field: {field_name}'.format(field_path_element.field_name)
                        print(f"\t\tOn field: {field_path_element.field_name}")
                error_message += '\n'

            campaign_data_errors.append({
                'account_id': account_id,
                'error_type': error_type,
                'error_message': error_message
            })
        except Exception as e:
            print("Search Response Error")
            print(e)
            campaign_data_errors.append({
                'account_id': account_id,
                'error_type': 'error',
                'error_message': f"'{e}'"
            })


        return {
            "account_id": account_id,
            "campaign_report": campaign_data_rows,
            "error_report": campaign_data_errors
        }


    global GADS_CREDENTIAL
    global GADS_API_VERSION
    global GCP_CREDENTIAL
    global GADS_SERVICE
    global GADS_CLIENT


    creds_gads = base64.b64decode(os.environ["SECRET_GADS_CREDS"])
    GADS_CREDENTIAL = json.loads(creds_gads)
    GADS_API_VERSION = "v8"


    GADS_CLIENT = GoogleAdsClient.load_from_dict(GADS_CREDENTIAL)
    GADS_CLIENT.login_customer_id: 6892170814
    GADS_SERVICE = GADS_CLIENT.get_service("GoogleAdsService", version=GADS_API_VERSION)

    print("API - Get Google Ads Account Tree")
    ah_customer_id = 6892170814
    ah_customer_id = str(ah_customer_id)



    GADS_FILTER_INTERVAL = ('DURING','LAST_7_DAYS')
    #GADS_FILTER_INTERVAL = ('BETWEEN',"'2020-11-15' AND '2020-11-16'")


    GADS_SERVICE = None
    GADS_SERVICE = GADS_CLIENT.get_service("GoogleAdsService", version=GADS_API_VERSION)
    GADS_CLIENT.login_customer_id: 6892170814

    bqte = utils.BigQueryTransferEtl(bq_credential)
    bqte.get_gcp_credential() # Setup service account credential for the project
    bqte.create_clients()  # Create the BigQuery clients used in the project

    # For INSERT
    Q = """
         SELECT  distinct account_id 
                 FROM `meli-marketing.COSTS.LK_GOOGLE_ADS_COST`
                 WHERE ACCOUNT_ROOT IN  ('MCC_MercadoLivre_Core','MCC_MercadoLibre_Core')
                    AND TIM_DAY > DATE_SUB(CURRENT_DATE, INTERVAL 30 DAY) 
        """
    bqQuery = bqte.bqClient.query(Q)

    accs = bqQuery.to_dataframe()

    all_lists = pd.DataFrame([])
    for acc in list(accs.account_id.values):
        campaign_reports = gads_campaign_report(str(acc), filter_interval=GADS_FILTER_INTERVAL)

        try:
            campaign_reports = pd.DataFrame(campaign_reports["campaign_report"])
            all_lists = all_lists.append(campaign_reports,ignore_index=True)
        except:
            pass


    ######################### Datos de campaña
    def gads_campaign_report(account_id, filter_interval=('DURING','LAST_7_DAYS')):
        global GADS_CLIENT
        global GADS_SERVICE

        enum_channel_types = GADS_CLIENT.get_type('AdvertisingChannelTypeEnum', version="v8").AdvertisingChannelType

        MAX_RETRY_ATTEMPTS = 3
        BACKOFF_FACTOR = 30
        BOOL_RESPONSE = False

        if filter_interval[0].upper() == 'DURING':
            campaign_data_range = "DURING {}".format(filter_interval[1])
        elif filter_interval[0].upper() == 'BETWEEN':
            campaign_data_range = "BETWEEN {}".format(filter_interval[1])


        today = date.today()
        yesterday = today - timedelta(days = 1)
        yesterday_str = yesterday.strftime("%Y-%m-%d")

        past = today - timedelta(days = 15)
        past_str = past.strftime("%Y-%m-%d")

        campaign_data_query = """
        SELECT
            campaign.name,
            campaign.id,
            campaign_criterion.criterion_id,
            segments.date,
            metrics.clicks,
            metrics.cost_micros,
            metrics.impressions,
            metrics.conversions_value_by_conversion_date,
            metrics.conversions_value,
            metrics.conversions,
            metrics.conversions_by_conversion_date
        FROM
            campaign_audience_view
        WHERE segments.date BETWEEN '"""+past_str+"""' AND '"""+yesterday_str+"""' 
            AND metrics.impressions>0 
        """

        campaign_data_rows = []
        campaign_data_errors = []
        cdr_append = campaign_data_rows.append
        retry_attempt = 0

        try:
            gads_response = GADS_SERVICE.search(customer_id = account_id, query=campaign_data_query)
            BOOL_RESPONSE = True

            for gads_row in gads_response:
                try:
                    campaign_data = {
                        "CAMPAIGN_NAME":gads_row.campaign.name,
                        "CAMPAIGN_ID":gads_row.campaign.id,
                        "CRITERION_ID":gads_row.campaign_criterion.criterion_id,
                        "DATE":gads_row.segments.date,
                        "CLICKS":gads_row.metrics.clicks,
                        "COST": gads_row.metrics.cost_micros,
                        "IMPRESSIONS":gads_row.metrics.impressions,
                        "CONV_VALUE_OCI": gads_row.metrics.conversions_value_by_conversion_date,
                        "CONV_VALUE": gads_row.metrics.conversions_value,
                        "CONVERSIONS": gads_row.metrics.conversions,
                        "CONVERSIONS_OCI": gads_row.metrics.conversions_by_conversion_date
                    }

                    cdr_append(campaign_data)

                except Exception as e:
                    print("Error occured handling response")
                    print(e)
                    continue

        except GoogleAdsException as ex:
            error_type = '{error_status}'.format(error_status=ex.error.code().name)
            error_messages = ''
            print(
                f"Request with ID '{ex.request_id}' failed with status "
                f"'{ex.error.code().name}' and includes the following errors:"
            )
            for error in ex.failure.errors:
                error_message = 'Error Message: {message}'
                print(f"\tError with message '{error.message}'.")
                if error.location:
                    error_message += ' - '
                    for field_path_element in error.location.field_path_elements:
                        error_message += '\n On Field: {field_name}'.format(field_path_element.field_name)
                        print(f"\t\tOn field: {field_path_element.field_name}")
                error_message += '\n'

            campaign_data_errors.append({
                'account_id': account_id,
                'error_type': error_type,
                'error_message': error_message
            })
        except Exception as e:
            print("Search Response Error")
            print(e)
            campaign_data_errors.append({
                'account_id': account_id,
                'error_type': 'error',
                'error_message': f"'{e}'"
            })


        return {
            "account_id": account_id,
            "campaign_report": campaign_data_rows,
            "error_report": campaign_data_errors
        }

    GADS_CREDENTIAL = json.loads(creds_gads)
    GADS_API_VERSION = "v8"


    GADS_CLIENT = GoogleAdsClient.load_from_dict(GADS_CREDENTIAL)
    GADS_CLIENT.login_customer_id: 6892170814
    GADS_SERVICE = GADS_CLIENT.get_service("GoogleAdsService", version=GADS_API_VERSION)

    print("API - Get Google Ads Account Tree")
    ah_customer_id = 6892170814
    ah_customer_id = str(ah_customer_id)



    GADS_FILTER_INTERVAL = ('DURING','LAST_7_DAYS')
    #GADS_FILTER_INTERVAL = ('BETWEEN',"'2020-11-15' AND '2020-11-16'")


    GADS_SERVICE = None
    GADS_SERVICE = GADS_CLIENT.get_service("GoogleAdsService", version=GADS_API_VERSION)
    GADS_CLIENT.login_customer_id: 6892170814

    all_metrics = pd.DataFrame([])
    for acc in list(accs.account_id.values):
        campaign_reports = gads_campaign_report(str(acc), filter_interval=GADS_FILTER_INTERVAL)

        try:
            campaign_reports = pd.DataFrame(campaign_reports["campaign_report"])
            all_metrics = all_metrics.append(campaign_reports,ignore_index=True)
        except:
            pass

    ######################### Mergeo todo
    metrics = pd.merge(all_metrics,all_lists.drop_duplicates(subset="CRITERION_ID"),on=[ "CRITERION_ID"])
    metrics["LIST_NAME"] = np.where(metrics.USER_LIST == '',metrics.USER_INTEREST_NAME,metrics.USER_LIST)
    metrics["LIST_TYPE"] = metrics["USER_TYPE"].replace({0:"USER_INTEREST",2:"APP_USERS",3:"COMBINED_LIST",5:"WEBSITE_VISITORS",6:"SIMILAR_AUDIENCE",7:"CUSTOMER_LIST"})

    metrics = metrics[["CAMPAIGN_NAME","DATE","CLICKS","COST","IMPRESSIONS","CONV_VALUE_OCI","CONV_VALUE",
               "CONVERSIONS","CONVERSIONS_OCI","LIST_NAME","LIST_TYPE"]]

    metrics = metrics.rename(columns = {"DATE":"FECHA"})
    metrics.to_csv("to_bq.csv",index=False)

    ######################### Subo a BQ
    from google.oauth2 import service_account
    from google.cloud import bigquery
    import time

    bqte = utils.BigQueryTransferEtl(bq_credential)
    bqte.get_gcp_credential() # Setup service account credential for the project
    bqte.create_clients()  # Create the BigQuery clients used in the project

    # Vacio la tabla
    bqQuery = bqte.bqClient.query("""
         DELETE FROM  meli-marketing.TEMP45.STAGING_AUDIENCE_TBL WHERE 1=1
    """)
    while(not bqQuery.done()): time.sleep(1)

    json_acct_info = json.loads(bq_credential)
    credentials = service_account.Credentials.from_service_account_info(json_acct_info)    
    client = bigquery.Client("meli-marketing", credentials)

    table_id = f"meli-marketing.TEMP45.STAGING_AUDIENCE_TBL"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV, skip_leading_rows=1, autodetect=True,
    )

    file_path = "to_bq.csv"
    print("Uploading to BigQuery")
    with open(file_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_id, job_config=job_config)

    while(not job.done()): time.sleep(1)

    bqte = utils.BigQueryTransferEtl(bq_credential)
    bqte.get_gcp_credential() # Setup service account credential for the project
    bqte.create_clients()  # Create the BigQuery clients used in the project

    bqQuery = bqte.bqClient.query("""
         DELETE FROM  meli-marketing.MODELLING.AUDIENCE_METRICS a
         WHERE EXISTS(SELECT 1 FROM meli-marketing.TEMP45.STAGING_AUDIENCE_TBL b where b.FECHA = a.FECHA and b.CAMPAIGN_NAME=b.CAMPAIGN_NAME)
    """)
    while(not bqQuery.done()): time.sleep(1)

    bqQuery = bqte.bqClient.query("""
         INSERT INTO meli-marketing.MODELLING.AUDIENCE_METRICS
         SELECT CAMPAIGN_NAME ,FECHA ,CLICKS,COST,IMPRESSIONS, CONV_VALUE_OCI, CONV_VALUE,CONVERSIONS,CONVERSIONS_OCI,LIST_NAME,LIST_TYPE 
         FROM meli-marketing.TEMP45.STAGING_AUDIENCE_TBL
    """)

    sub.subscriber('end', "ALL", "ML", "AUDIENCIAS", "REPORTING")
    
except Exception as inst:
    print(type(inst))    
    print(inst.args)    
    print(inst)  
    subject = 'Falla en Load audiencias'
    body = 'Fallo la función de load de audiencias'
    sub = utils.Substatus("", mail_to='pedro.zenone@mercadolibre.com')
    sub.send_alert(subject, body)

API - Get Google Ads Account Tree
API - Get Google Ads Account Tree
Uploading to BigQuery
